In [1]:
%load_ext autoreload

In [2]:
%autoreload

import h2o
import create_data as create
import pandas as pd
import utilities as utils

from h2o.automl import H2OAutoML

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data = create.CreateTrainingData()
train = data.create_training_data()
test = create.create_test_data()
target = create.create_target_data()

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /home/stinky/.local/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpbhwhr5wi
  JVM stdout: /tmp/tmpbhwhr5wi/h2o_stinky_started_from_python.out
  JVM stderr: /tmp/tmpbhwhr5wi/h2o_stinky_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.4
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_stinky_5b5u3f
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.889 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [5]:
# Delete all rows with nans in pv_measurement column in train
train = train[train['pv_measurement'].isna() == False]

X_train = h2o.H2OFrame(train)
y = 'pv_measurement'

X = X_train.columns
X.remove(y)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
autoML = H2OAutoML(max_models=30, seed=1, nfolds=5)
autoML.train(x=X, y=y, training_frame=X_train)

In [ ]:
# Remove features that are not in the training set
test_pd = test[X]

# Set nans to 0 in test_pd
# test_pd.fillna(0, inplace=True)

test = h2o.H2OFrame(test_pd)

prediction = autoML.predict(test)

In [ ]:
preds_df = prediction.as_data_frame()
preds_df[preds_df < 0] = 0

results = pd.concat([test_pd.reset_index(), preds_df], axis=1)
results['date_forecast'] = pd.to_datetime(results['date_forecast'])
results.set_index('date_forecast', inplace=True)
results.rename(columns={'predict': 'pv_prediction'}, inplace=True)

utils.save_to_csv(results, 'h2o_nans_to_0')